<a href="https://colab.research.google.com/github/mihirdeo16/jubilant-memory/blob/master/Deep%2520Learning/HF_NLP_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!nvidia-smi

Fri Apr 14 18:22:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding, AutoModelForSequenceClassification
from datasets import load_dataset, load_dataset_builder
from torch.utils.data import DataLoader
from datasets import list_metrics, load_metric
from transformers import AdamW
import torch
import evaluate
from tqdm.auto import tqdm

In [4]:
ds = load_dataset("rotten_tomatoes")
metric = evaluate.load("accuracy")

model_config = 'bert-base-uncased'#"distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(model_config, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_config)

  0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_text(df):
    return tokenizer(df['text'],truncation=True)

ds_tokenize = ds.map(tokenize_text,batched=True,batch_size=1000)
ds_tokenize = ds_tokenize.rename_column("label", "labels")
ds_tokenize = ds_tokenize.remove_columns(["text"])
ds_tokenize.set_format("torch")
ds_tokenize["train"].column_names

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [6]:
train_dl = DataLoader(
    ds_tokenize['train'],shuffle=True,batch_size=8,collate_fn=data_collator
)

eval_dl = DataLoader(
    ds_tokenize['validation'],batch_size=8,collate_fn=data_collator
)

test_dl = DataLoader(
    ds_tokenize['test'],batch_size=8,collate_fn=data_collator
)

In [7]:
for batch in train_dl:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 41]),
 'token_type_ids': torch.Size([8, 41]),
 'attention_mask': torch.Size([8, 41])}

In [8]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

sample_predict = torch.argmax(outputs.logits,dim =-1)
metric.compute(predictions=sample_predict,references=batch["labels"])

tensor(0.6551, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


{'accuracy': 0.625}

In [9]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [10]:
from transformers import get_scheduler

from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
progress_bar = tqdm(range(num_training_steps))
epochs_bar = tqdm(range(num_epochs))

for epoch in range(num_epochs):
    model.train()
    for batch in train_dl:
        batch = {k:v.to(device) for k,v in batch.items()}

        output = model(**batch)
        loss = output.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)

    model.eval()
    for batch in eval_dl:
        batch = {k:v.to(device) for k,v in batch.items()}
        with torch.no_grad():
            prediction = model(**batch)

        eval_prediction = torch.argmax(prediction.logits,dim=-1)
        metric.add_batch(predictions=eval_prediction,references=batch["labels"]) 
    epochs_bar.update(1)

    print(epoch,"-- Accuracy:",metric.compute()["accuracy"])

  0%|          | 0/3201 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0 -- Accuracy: 0.849906191369606
1 -- Accuracy: 0.8555347091932458
2 -- Accuracy: 0.8545966228893058


In [12]:
model.eval()

for batch in test_dl:
    batch = {k:v.to(device) for k,v in batch.items()}
    with torch.no_grad():
        prediction = model(**batch)
    test_predict = torch.argmax(prediction.logits,dim =-1)
    metric.add_batch(predictions=test_predict,references=batch["labels"])
metric.compute()

{'accuracy': 0.8545966228893058}

### END